In [1]:
# self-generated similar design?
import pandas as pd
import numpy as np
import pickle

In [2]:
# 1638 words
with open("../Data/wordpools/wasnorm_wordpool.txt") as f:
    items = f.readlines()
items = [i.replace("\n","") for i in items]
items = np.array(items)

In [3]:
items[-2]

'ZOO'

In [4]:
nsubj = 100
nlist = 3
simu_sess_num = nsubj * nlist
n = 25
v = 5
n_exitem = 15
n_expair = 15
extra = 2 * (n_exitem * 1 + n_expair * 2)
wordpool = np.arange(1,1639)

In [5]:
df_study = pd.DataFrame()
df_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for subj in range(nsubj):

    all_words = rng.choice(wordpool, nlist*(2*(n+v) + extra), replace=False)

    for lst in range(nlist):

        tmp_df_study = pd.DataFrame()
        tmp_df_test1 = pd.DataFrame()
        tmp_df_test2 = pd.DataFrame()
        
        # study
        pres_words = all_words[lst*(2*(n+v) + extra):lst*(2*(n+v) + extra)+2*n]
        pres_words = np.reshape(pres_words, (n,2))
        tmp_df_study['study_itemno1'] = pres_words[:,0]
        tmp_df_study['study_itemno2'] = pres_words[:,1]
        tmp_df_study['study_item1'] = items[pres_words[:,0]-1]
        tmp_df_study['study_item2'] = items[pres_words[:,1]-1]
        tmp_df_study['pair_idx'] = np.arange(n) + lst*(n+v)
        tmp_df_study['list'] = lst
        tmp_df_study['session'] = subj
        tmp_df_study['subject'] = subj
        # df_study = pd.concat([df_study,tmp_df_study])

        # repeat 6 times, no continuous
        idx_order = []
        repeat_cnt = np.zeros(24)
        number_left = np.arange(24)
        last_pick = 0
        for i in range(24*6):
            pick = rng.choice(number_left)

            loop_cnt = 0
            while pick == last_pick:
                pick = rng.choice(number_left)
                loop_cnt += 1
                if loop_cnt > 100:
                    print("fail")
                    break

            idx_order.append(pick)
            repeat_cnt[pick] += 1
            if repeat_cnt[pick] == 6:
                np.delete(number_left, pick)
            last_pick = pick
        tmp_df_study = tmp_df_study.iloc[idx_order]
        df_study = pd.concat([df_study,tmp_df_study])

        # test conditions
        valid_words = all_words[lst*(2*(n+v) + extra):lst*(2*(n+v) + extra)+2*(n+v)]
        valid_words = np.reshape(valid_words, (n+v,2))
        conditions = ["Same_Item"] * 6 + ["Different_Item"] * 6 + ["Intact_Pair"] * 3 + ["Item_Pair"] * 9 + ["Pair_Item"] * 6
        conditions = rng.permutation(conditions)

        test1_probe = []
        test2_probe = []
        for i, c in enumerate(conditions):
            order = rng.choice([0,1])
            this_pair = valid_words[i].tolist()
            if c == "Same_Item":
                test1_probe.append([this_pair[order],-1])
                test2_probe.append([this_pair[order],-1])
            elif c == "Different_Item":
                test1_probe.append([this_pair[order],-1])
                test2_probe.append([this_pair[1-order],-1])
            elif c == "Intact_Pair":
                test1_probe.append(this_pair)
                test2_probe.append(this_pair)
            elif c == "Item_Pair":
                test1_probe.append([this_pair[order],-1])
                test2_probe.append(this_pair)
            elif c == "Pair_Item":
                test1_probe.append(this_pair)
                test2_probe.append([this_pair[order],-1])
        test1_probe = np.array(test1_probe)
        test2_probe = np.array(test2_probe)

        # test1
        extra_words = all_words[lst*(2*(n+v) + extra)+2*(n+v):lst*(2*(n+v) + extra)+2*(n+v)+int(extra/2)]
        extra_items = extra_words[0:n_exitem]
        extra_pairs = np.reshape(extra_words[n_exitem:], (n_expair,2))
        tmp_df_test1['test_itemno1'] = np.concatenate((test1_probe[:,0], extra_items, extra_pairs[:,0]))
        tmp_df_test1['test_itemno2'] = np.concatenate((test1_probe[:,1], [-1]*n_exitem, extra_pairs[:,1]))
        tmp_df_test1['test_item1'] = [items[i-1] if i != -1 else 'None' for i in tmp_df_test1['test_itemno1']]
        tmp_df_test1['test_item2'] = [items[i-1] if i != -1 else 'None' for i in tmp_df_test1['test_itemno2']]
        tmp_df_test1['correct_ans'] = [1] * n + [0] * (v + n_exitem + n_expair)
        tmp_df_test1['pair_idx'] = np.concatenate((np.arange(n+v)+lst*(n+v),[-1] * (n_exitem + n_expair)))
        tmp_df_test1['type'] =  np.concatenate((conditions, ["extra"] * (n_exitem + n_expair)))
        tmp_df_test1['test'] = 1
        tmp_df_test1['list'] = lst
        tmp_df_test1['session'] = subj
        tmp_df_test1['subject'] = subj
        tmp_df_test1 = tmp_df_test1.iloc[rng.permutation(tmp_df_test1.index)].reset_index(drop=True)

        # test2
        extra_words = all_words[(lst+1)*(2*(n+v) + extra)-int(extra/2):(lst+1)*(2*(n+v) + extra)]
        extra_items = extra_words[0:n_exitem]
        extra_pairs = np.reshape(extra_words[n_exitem:], (n_expair,2))
        tmp_df_test2['test_itemno1'] = np.concatenate((test2_probe[:,0], extra_items, extra_pairs[:,0]))
        tmp_df_test2['test_itemno2'] = np.concatenate((test2_probe[:,1], [-1]*n_exitem, extra_pairs[:,1]))
        tmp_df_test2['test_item1'] = [items[i-1] if i != -1 else 'None' for i in tmp_df_test2['test_itemno1']]
        tmp_df_test2['test_item2'] = [items[i-1] if i != -1 else 'None' for i in tmp_df_test2['test_itemno2']]
        tmp_df_test2['correct_ans'] = [1] * n + [0] * (v + n_exitem + n_expair)
        tmp_df_test2['pair_idx'] = np.concatenate((np.arange(n+v)+lst*(n+v),[-1] * (n_exitem + n_expair)))
        tmp_df_test2['type'] = np.concatenate((conditions, ["extra"] * (n_exitem + n_expair)))
        tmp_df_test2['test'] = 2
        tmp_df_test2['list'] = lst
        tmp_df_test2['session'] = subj
        tmp_df_test2['subject'] = subj
        tmp_df_test2 = tmp_df_test2.iloc[rng.permutation(tmp_df_test2.index)].reset_index(drop=True)

        df_test = pd.concat([df_test,tmp_df_test1,tmp_df_test2])

df_study.reset_index(inplace = True, drop=True)
df_test.reset_index(inplace = True, drop=True)  

In [6]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,pair_idx,list,session,subject
0,1026,897,PATRIOT,MINK,14,0,0,0
1,1524,1231,TUTU,SANDPAPER,0,0,0,0
2,893,437,MILDEW,DANCER,1,0,0,0
3,1492,1034,TOY,PEARL,5,0,0,0
4,1556,1586,VENT,WAND,21,0,0,0
...,...,...,...,...,...,...,...,...
43195,34,968,APPLICATION,OATS,72,2,99,99
43196,1165,250,RAM,CART,71,2,99,99
43197,600,1144,FRAME,PUPPY,60,2,99,99
43198,345,1086,COLLAR,PLATE,73,2,99,99


In [7]:
df_test

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,pair_idx,type,test,list,session,subject
0,103,-1,BEAD,None,0,-1,extra,1,0,0,0
1,809,1001,LEG,PAGE,1,16,Intact_Pair,1,0,0,0
2,51,-1,ASTRONAUT,None,1,13,Item_Pair,1,0,0,0
3,1512,-1,TRUCK,None,0,-1,extra,1,0,0,0
4,1026,-1,PATRIOT,None,1,14,Item_Pair,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
35995,734,1328,IGLOO,SLOPE,0,87,Intact_Pair,2,2,99,99
35996,886,-1,MEDICINE,None,0,-1,extra,2,2,99,99
35997,1088,-1,PLAZA,None,0,-1,extra,2,2,99,99
35998,939,-1,NECK,None,1,69,Same_Item,2,2,99,99


In [8]:
# save
with open('../Data/simuS2_design.pkl', 'wb') as outp:
    pickle.dump(df_study, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(df_test, outp, pickle.HIGHEST_PROTOCOL)